<div class="alert alert-block alert-info">  
<strong>Hello! I am a new Kaggler. I start to study the machine learning. As a result, I look at other notebook about this problem and conclude it according to my understanding.


<div class="alert alert-block alert-info">
    <h2><strong><center>Reference</center><strong></h2>

1. [Simple EDA](https://www.kaggle.com/subinium/tps-oct-simple-eda/comments)
2. [🔥TensorFlow Decision Forests 🌵 🎄 🌲 🌳 + W&B](https://www.kaggle.com/usharengaraju/tensorflow-decision-forests-w-b)
3. [TPS Oct 2021: single LightGBM](https://www.kaggle.com/hiro5299834/tps-oct-2021-single-lightgbm)
4. [LightGBM Baseline](https://www.kaggle.com/yukiyamamoto/lightgbm-baseline)
5. [TPS Sep 2021 single LGBM](https://www.kaggle.com/hiro5299834/tps-sep-2021-single-lgbm)

<div class="alert alert-block alert-info">
    <h2><strong><center>Import Library</center><strong></h2>

In [ ]:
#data process
import pandas as pd
import numpy as np

#pandas setting
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_info_columns", 300)

In [ ]:
#data visualiztion
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#machine learning model
from sklearn.preprocessing import RobustScaler
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

<div class="alert alert-block alert-info">
    <h2><strong><center>Overview</center><strong></h2>

### Loading the data

Raw data is too large. We had better sample 10k rows to analyze.

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2021/train.csv', nrows=10000)
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2021/test.csv', nrows=10000)

In [ ]:
#print(train_df.head())
#print(train_df.info())
#print(train_df.columns)

### Check the feature type
Usually, "feature.dtype = int" means this feature is discrete and in this problem it represent "binary". Meanwhile, "object" means string, "float" means continous.

In [ ]:
features = [col for col in train_df.columns if 'f' in col]

In [ ]:
binary_feature = []
continuous_feature = [] 
for feature in features:
    if "int" in str(train_df[feature].dtype):   #int64, int32... belong to int.
        binary_feature.append(feature)
    else:
        continuous_feature.append(feature)

In [ ]:
print("*"*50)
print("the number of continuous feature:  {}".format(len(continuous_feature)))
print("*"*50)
print("\n")
print("*"*50)
print("the number of binary feature:  {}".format(len(binary_feature)))
print("*"*50)
print("\n")

In [ ]:
train_df[features].describe().T.style.bar(subset = ["mean"]).background_gradient(subset = ["std"]).background_gradient(subset = ["50%"])

### Missing value

In [ ]:
print("*"*50)
print("the number of missing value in train sample is {0}".format(train_df[features].isnull().sum().sum()))
print("*"*50)
print("\n")
print("*"*50)
print("the number of missing value in test sample is {0}".format(test_df[features].isnull().sum().sum()))
print("*"*50)
print("\n")

### Target

In [ ]:
train_df["target"].value_counts().plot(kind = "pie", figsize = (8,8)).legend()

### Dataset conclusion
According to the above result, we can find that:
1. **missing value.** the trainset and testset all don't have the missing data. so we don't need to complete.
2. **dataset size.** the dataset have 284 features. train dataset has 1000k rows and test dataset has 500k rows. it's large and maybe we should convert or delete some features to accelerate the training process
3. **feature type.** type of feature can be classified to continuous feature and binary feature.
4. **target.** The value of target is 0 or 1. And the distribution of target is normal

<div class="alert alert-block alert-info">
    <h2><strong><center>Data Analysis</center><strong></h2>

Too many features! I learnt some method about data analysis in GetStarted competition such as Titanic. Howeve , these methods are used to analyze the relation between one feature and target. if I analyze every feature using these methods, the work is hard.

### Observe the corvariance 
let us observe the corvariance between the feature and target. it is important to find the main feature quickly. Maybe this dataset have one feature that have non-linear relationship such as Y=X^2 and its corvariance equals zero so we may overlook this feature using this data analyis method. But this kind of method can find the linear relationship.

In [ ]:
corr_matrix = train_df[features + ["target"]].corr()

In [ ]:
feature_corr_target = corr_matrix["target"].apply(lambda x:abs(x)).sort_values(ascending = False)[0:30]
feature_corr_target.drop("target", inplace = True)
plt.figure(figsize = (15,7))
sns.barplot(x = feature_corr_target.index, y = feature_corr_target)

### Data visualization


### kdeplot - find the contribution of continuous feature

In [ ]:
fig, axes = plt.subplots(24,10,figsize=(17, 40))
axes = axes.flatten()

index = 0 

for idx, ax in enumerate(axes):
    if len(continuous_feature) == index:
        break
    sns.kdeplot(data=train_df, x=continuous_feature[index],hue = 'target', ax=ax)
    ax.set_title(continuous_feature[index], loc='right', weight='bold', fontsize=10)
    index = index + 1

### histplot - find the contribution of binary feature

In [ ]:
fig, axes = plt.subplots(12,4,figsize=(12,30))
axes = axes.flatten()

index = 0
for idx,ax in enumerate(axes):
    if index == len(binary_feature):
        break
    sns.histplot(data = train_df, x= binary_feature[index], hue = "target", ax = ax, multiple="dodge")
    index = index + 1

According to these histplot, we find that most of the binary feature have a little influence on target independently except "f22". Also, the correlation coefficient of f22 is -0.5139, this influence is obvious.


### Univariate analysis
Emmm, we have plot about each feature and target. In fact, most of the feature influence target slightly.

<div class="alert alert-block alert-info">
    <h2><strong><center>Training and Predicting</center><strong></h2> 

### Dataset

In [ ]:
train_dataset = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/train.csv")
test_dataset = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/test.csv")

In [ ]:
scaler = RobustScaler()
train_dataset[features] = scaler.fit_transform(train_dataset[features])
test_dataset[features] = scaler.transform(test_dataset[features])

### Parameters  - LightGBM Hyperparameters

In [ ]:
N_SPLITS = 5
N_ESTIMATORS = 20000  
EARLY_STOPPING_ROUNDS = 1000 
VERBOSE = 1000
SEED = 200

In [ ]:
lgb_params = {
    'objective': 'binary',
    'n_estimators': N_ESTIMATORS,
    'random_state': SEED,
    'learning_rate': 5e-3,
    'subsample': 0.6,
    'subsample_freq': 1,
    'colsample_bytree': 0.4,
    'reg_alpha': 10.0,
    'reg_lambda': 1e-1,
    'min_child_weight': 256,
    'min_child_samples': 20,
}

### Training Process

**scale** is used to weight the model.

In [ ]:
scale = [0.1, 0.1, 0.2, 0.2, 0.4]

In [ ]:
kdf = StratifiedKFold(n_splits = N_SPLITS, shuffle = True, random_state = SEED)
y_pred = np.zeros(test_dataset.shape[0])
lgb_importance = pd.DataFrame()

pred_df = pd.DataFrame()
valid_accuracy = pd.Series()

for fold, (train_idx, valid_idx) in enumerate(kdf.split(train_dataset,train_dataset["target"])):
    print(f'====== Fold {fold} ======')
    X_train = train_dataset[features].iloc[train_idx]
    y_train = train_dataset["target"].iloc[train_idx]
    X_valid = train_dataset[features].iloc[valid_idx]
    y_valid = train_dataset["target"].iloc[valid_idx]
    model = LGBMClassifier(**lgb_params).fit(X_train, y_train,
                                            eval_set = [(X_valid, y_valid)],
                                            eval_metric = "auc",
                                            early_stopping_rounds = EARLY_STOPPING_ROUNDS,
                                            verbose = VERBOSE)
    pred_df[str(fold)] = model.predict_proba(test_dataset[features])[:,-1] / N_SPLITS
    valid_accuracy[str(fold)] = roc_auc_score(y_valid, model.predict(X_valid))
    lgb_importance["fold_"+str(fold)] = model.feature_importances_
lgb_importance["name"] = model.feature_name_


In [ ]:
ensemble_sort = valid_accuracy.argsort()
for i in range(5):
    y_pred += pred_df[str(ensemble_sort[i])] * scale[i] 

<div class="alert alert-block alert-info">
    <h2><strong><center>Model Analysis</center><strong></h2>

In [ ]:
lgb_importance["mean"] = lgb_importance[["fold_0", "fold_1"]].mean(axis = 1)

In [ ]:
importance_result = lgb_importance[["name", "mean"]].sort_values(by = "mean", ascending = False)[:30]

plt.figure(figsize = (15,7))

sns.barplot(data = importance_result, x = "name", y = "mean")


<div class="alert alert-block alert-info">
    <h2><strong><center>Submission</center><strong></h2>

In [ ]:
submission = pd.DataFrame({"id": test_dataset["id"], "target": y_pred})
submission.to_csv("submission.csv", index = False)

<div class="alert alert-block alert-info">
    <h2><strong><center>Experiment</center><strong></h2>

1. lightgbm without params: valid 0.849536 score 0.84863
2. lightgbm with KFold and params: 